# 판례 키워드 추출
- 판례 검색의 성능 향상을 위해 판례의 키워드를 추출한다.
    - 키워드 추출에는 FastText 모델이 사용된다.

## 1. 패키지 로드

In [1]:
import pandas as pd
import numpy as np
import itertools

from collections import Counter
from konlpy.tag import Mecab
from gensim import models
from tqdm.notebook import tqdm
from sklearn.metrics.pairwise import cosine_similarity

## 2. 데이터 로드

In [2]:
prec_df = pd.read_csv("./data/prec_data_refine.csv", encoding='utf-8-sig')
prec_df.head()

C:\Users\Park Ji Seong\anaconda3\envs\capstone\lib\site-packages\IPython\core\interactiveshell.py:3552: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,판례일련번호,사건명,사건번호,선고일자,선고,법원명,법원종류코드,사건종류명,사건종류코드,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용
0,220283,미성년자의제강간,2021노824,2022-04-15,선고,수원고법,400202,형사,400102,판결 : 확정,"피고인이 카카오톡 오픈채팅방을 통해 미성년자 甲(여, 11세)을 알게 된 후 당...","피고인이 카카오톡 오픈채팅방을 통해 미성년자 甲(여, 11세)을 알게 된 후 당...",구 형법(2020. 5. 19. 법률 제17265호로 개정되기 전의 것) 제305조...,없음,【피 고 인】 피고인【항 소 인】 쌍방【검 사】 이지은 외 1인【변 호 인】 ...
1,220263,소송비용액확정,2022브2061,2022-04-11,자,서울고등법원,400202,가사,400103,결정,미기재,미기재,"가사소송법 제37조의2, 제63조 제1항, 가사소송규칙 제95조 제1항, 구 변호사...",없음,"【신청인, 피항고인】 신청인【피신청인, 항고인】 피신청인【제1심결정】 서울가정법원 ..."
2,220249,손해배상(기),2021나24173,2022-03-23,선고,대구고법,400202,민사,400101,판결 : 확정,甲이 乙을 상대로 소송을 제기하자 乙이 丙과 소송위임계약을 체결하여 丙이 乙을 ...,甲이 乙을 상대로 소송을 제기하자 乙이 丙과 소송위임계약을 체결하여 丙이 乙을 ...,"민법 제390조, 제681조, 민사소송법 제225조, 제226조, 제231조",없음,"【원고, 항소인 겸 피항소인】 원고 (소송대리인 법무법인 우정 담당변호사 김병구 외..."
3,220261,소송비용액확정,2020스507,2022-03-22,자,대법원,400201,가사,400103,결정,마류 가사비송사건에서 변호사보수가 절차비용에 산입되는지 여부(적극),미기재,"가사소송법 제37조의2, 가사소송규칙 제95조 제1항, 구 변호사보수의 소송비용 산...",없음,"【신청인(재심상대방), 재항고인】 신청인(재심상대방) (소송대리인 법무법인(유한) ..."
4,220241,손해배상(의)[환자가 치료 도중 뇌출혈로 사망하자 의료과실에 의한 손해배상을 구하는...,2018다263434,2022-03-17,선고,대법원,400201,민사,400101,판결,의사가 의료행위를 할 때 취하여야 할 주의의무의 정도 및 기준 / 특히 환자가 ...,의사가 진찰·치료 등의 의료행위를 할 때에는 사람의 생명·신체·건강을 관리하는 ...,"민법 제390조, 제750조\n","대법원 2018. 11. 29. 선고 2016다266606, 266613 판결(공2...","【원고, 상고인】 원고 1 외 2인 (소송대리인 변호사 신현호 외 4인)【피고, 피..."


In [3]:
prec_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83318 entries, 0 to 83317
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   판례일련번호  83318 non-null  int64 
 1   사건명     83318 non-null  object
 2   사건번호    83318 non-null  object
 3   선고일자    83318 non-null  object
 4   선고      83318 non-null  object
 5   법원명     83318 non-null  object
 6   법원종류코드  83318 non-null  object
 7   사건종류명   83318 non-null  object
 8   사건종류코드  83318 non-null  object
 9   판결유형    83318 non-null  object
 10  판시사항    83318 non-null  object
 11  판결요지    83318 non-null  object
 12  참조조문    83318 non-null  object
 13  참조판례    83314 non-null  object
 14  판례내용    83318 non-null  object
dtypes: int64(1), object(14)
memory usage: 9.5+ MB


In [4]:
# 전처리 개선전 임시 코드
prec_df['참조판례'] = prec_df['참조판례'].fillna(value='없음')

## 3. 판례 전처리

### 1) 불용어 선정을 위한 명사 추출

In [5]:
# 1. Mecab load
mecab = Mecab('C:\mecab\mecab-ko-dic')

In [6]:
# 2. 판시사항, 판결요지, 판례내용 합치기
df_X = []

for idx in tqdm(range(len(prec_df))):
    if prec_df['판시사항'][idx] == prec_df['판결요지'][idx]:
        x = prec_df['판결요지'][idx] + prec_df['판례내용'][idx]
    else:
        x = prec_df['판시사항'][idx] + prec_df['판결요지'][idx] + prec_df['판례내용'][idx]
    df_X.append(x)

  0%|          | 0/83318 [00:00<?, ?it/s]

In [7]:
# 3. 데이터프레임화
df_X = pd.DataFrame(columns=['X'], data=df_X)
df_X.head()

,X
0,"피고인이 카카오톡 오픈채팅방을 통해 미성년자 甲(여, 11세)을 알게 된 후 당..."
1,"미기재【신청인, 피항고인】 신청인【피신청인, 항고인】 피신청인【제1심결정】 서울가정..."
2,甲이 乙을 상대로 소송을 제기하자 乙이 丙과 소송위임계약을 체결하여 丙이 乙을 ...
3,마류 가사비송사건에서 변호사보수가 절차비용에 산입되는지 여부(적극)미기재【신청인(재...
4,의사가 의료행위를 할 때 취하여야 할 주의의무의 정도 및 기준 / 특히 환자가 ...


In [8]:
# 4. 한글을 제외하고 모두 제거
df_X['X'] = df_X['X'].str.replace('[^가-힣 ]', '')

C:\Users\Park Ji Seong\anaconda3\envs\capstone\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [9]:
# 5. 명사 추출
nouns_list = []

for x in tqdm(df_X['X']):
    nouns = mecab.nouns(x)
    nouns_list.append(nouns)

  0%|          | 0/83318 [00:00<?, ?it/s]

In [10]:
# 4. 리스트 1차원으로 만들기
nouns_flatten = [y for x in nouns_list for y in x]

In [14]:
# 5. 카운트
nouns_count = Counter(nouns_flatten)

In [12]:
# 6. 불용어 선정 : 단어 길이 2미만, 등장횟수 9만번 이상
stopword = []

for word, count in tqdm(nouns_count.most_common(5000)):
    if (count >= 90000) | (len(word) < 2):
        stopword.append(word)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [15]:
# 7. 불용어를 제외하고 명사 추출
refined_nouns_list = []

for nouns in tqdm(nouns_list):
    temp_nouns_list = []
    for noun in nouns:
        if (noun not in stopword) and (len(noun) >= 2):
            temp_nouns_list.append(noun)
    temp_nouns_list = list(set(temp_nouns_list))
    refined_nouns_list.append(temp_nouns_list)

  0%|          | 0/83318 [00:00<?, ?it/s]

### 2) 키워드 추출

In [16]:
# 1. 모델 로드
fasttext_model = models.fasttext.load_facebook_model('./fasttext/cc.ko.300.bin')

In [17]:
# 2. 키워드 추출을 위한 Max Sum Similarity 함수 정의
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n=5, nr_candidates=10):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [words[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [18]:
# 3. 단어(명사) 임베딩
prec_keywords = []
candidate_embeddings = []

for nouns in tqdm(refined_nouns_list):
    candidate_embedding = []
    for noun in nouns:
        candidate_embedding.append(fasttext_model.wv.get_vector(noun))
    candidate_embeddings.append(candidate_embedding)

  0%|          | 0/83318 [00:00<?, ?it/s]

In [19]:
%%time
# 4. 키워드 추출
for i in tqdm(range(len(refined_nouns_list))):
    # 문장 임베딩
    doc_embedding = fasttext_model.wv.get_sentence_vector(df_X['X'][i])
    
    # 단어 임베딩
    candidate_embedding = fasttext_model.wv.get_mean_vector(refined_nouns_list[i])
    
    if len(refined_nouns_list[i]) < 20:
        keywords = max_sum_sim(doc_embedding.reshape(1, -1), candidate_embeddings[i], refined_nouns_list[i], 5, len(refined_nouns_list[i]))
    else:
        keywords = max_sum_sim(doc_embedding.reshape(1, -1), candidate_embeddings[i], refined_nouns_list[i], 5, 20)
    prec_keywords.append(keywords)

  0%|          | 0/83318 [00:00<?, ?it/s]

Wall time: 2h 55min 54s


## 4. 키워드 저장

In [20]:
join_list = []

for keywords in tqdm(prec_keywords):
    join_keywords = ', '.join(keywords)
    join_list.append(join_keywords)

  0%|          | 0/83318 [00:00<?, ?it/s]

In [21]:
prec_df['키워드'] = join_list

In [22]:
prec_df.head()

,판례일련번호,사건명,사건번호,선고일자,선고,법원명,법원종류코드,사건종류명,사건종류코드,판결유형,판시사항,판결요지,참조조문,참조판례,판례내용,키워드
0,220283,미성년자의제강간,2021노824,2022-04-15,선고,수원고법,400202,형사,400102,판결 : 확정,"피고인이 카카오톡 오픈채팅방을 통해 미성년자 甲(여, 11세)을 알게 된 후 당...","피고인이 카카오톡 오픈채팅방을 통해 미성년자 甲(여, 11세)을 알게 된 후 당...",구 형법(2020. 5. 19. 법률 제17265호로 개정되기 전의 것) 제305조...,없음,【피 고 인】 피고인【항 소 인】 쌍방【검 사】 이지은 외 1인【변 호 인】 ...,"미연, 제공, 자의, 그것, 미만"
1,220263,소송비용액확정,2022브2061,2022-04-11,자,서울고등법원,400202,가사,400103,결정,미기재,미기재,"가사소송법 제37조의2, 제63조 제1항, 가사소송규칙 제95조 제1항, 구 변호사...",없음,"【신청인, 피항고인】 신청인【피신청인, 항고인】 피신청인【제1심결정】 서울가정법원 ...","청인, 규칙, 첨부, 원임, 취하"
2,220249,손해배상(기),2021나24173,2022-03-23,선고,대구고법,400202,민사,400101,판결 : 확정,甲이 乙을 상대로 소송을 제기하자 乙이 丙과 소송위임계약을 체결하여 丙이 乙을 ...,甲이 乙을 상대로 소송을 제기하자 乙이 丙과 소송위임계약을 체결하여 丙이 乙을 ...,"민법 제390조, 제681조, 민사소송법 제225조, 제226조, 제231조",없음,"【원고, 항소인 겸 피항소인】 원고 (소송대리인 법무법인 우정 담당변호사 김병구 외...","인피, 취하, 그것, 방안, 미만"
3,220261,소송비용액확정,2020스507,2022-03-22,자,대법원,400201,가사,400103,결정,마류 가사비송사건에서 변호사보수가 절차비용에 산입되는지 여부(적극),미기재,"가사소송법 제37조의2, 가사소송규칙 제95조 제1항, 구 변호사보수의 소송비용 산...",없음,"【신청인(재심상대방), 재항고인】 신청인(재심상대방) (소송대리인 법무법인(유한) ...","미기, 담당, 영향, 개정, 용액"
4,220241,손해배상(의)[환자가 치료 도중 뇌출혈로 사망하자 의료과실에 의한 손해배상을 구하는...,2018다263434,2022-03-17,선고,대법원,400201,민사,400101,판결,의사가 의료행위를 할 때 취하여야 할 주의의무의 정도 및 기준 / 특히 환자가 ...,의사가 진찰·치료 등의 의료행위를 할 때에는 사람의 생명·신체·건강을 관리하는 ...,"민법 제390조, 제750조\n","대법원 2018. 11. 29. 선고 2016다266606, 266613 판결(공2...","【원고, 상고인】 원고 1 외 2인 (소송대리인 변호사 신현호 외 4인)【피고, 피...","두피, 참조, 구토, 사람, 정확"


In [23]:
prec_df.to_csv('./data/prec_data_keyword.csv', encoding='utf-8-sig', index=False)